# 1. Setup

## 1.1 Module import

Import necessary modules

In [1]:
import tensorflow as tf
import keras
import dataset_utils

2024-05-21 13:16:24.537624: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-21 13:16:24.866950: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 13:16:24.867012: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 13:16:24.874233: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-21 13:16:24.976067: I tensorflow/core/platform/cpu_feature_guar

## 1.2 Dataset import using opencv

Define parameters for the image import

In [2]:
img_width = 100
img_height = 40

train_dataset_path = r'/mnt/d/Dev/DHBW/DHBW_Studienarbeit/training_images/training_dataset/all_cropped'
validation_dataset_path = r'/mnt/d/Dev/DHBW/DHBW_Studienarbeit/training_images/validation_dataset/all_cropped'

Import the data for the training dataset

In [3]:
train_labels, train_images = dataset_utils.load_dataset(path_to_dataset = train_dataset_path, change_contrast = True, grayscale = True, image_size = (img_width, img_height))

Loading 18777 images and labels. This might take a while, please wait...
Progress 100 %
Loaded 18777 images and 18777 labels from directory /mnt/d/Dev/DHBW/DHBW_Studienarbeit/training_images/training_dataset/all_cropped
Loaded images have shape (40, 100, 3).


Import the data for the validation dataset

In [4]:
validation_labels, validation_images = dataset_utils.load_dataset(path_to_dataset = validation_dataset_path, change_contrast = True, grayscale = True, image_size = (img_width, img_height))

Loading 1812 images and labels. This might take a while, please wait...
Progress 100 %
Loaded 1812 images and 1812 labels from directory /mnt/d/Dev/DHBW/DHBW_Studienarbeit/training_images/validation_dataset/all_cropped
Loaded images have shape (40, 100, 3).


# 2. Transfer learning

## 2.1 Construct the new model

Load the base model

In [5]:
IMG_SIZE = (img_height, img_width)
IMG_SHAPE = IMG_SIZE + (3,)
print(IMG_SHAPE)

base_model = tf.keras.applications.MobileNetV3Small(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False

(40, 100, 3)


2024-05-21 13:18:25.642082: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-21 13:18:25.906989: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-21 13:18:25.907079: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-21 13:18:25.909220: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-21 13:18:25.909278: I external/local_xla/xla/stream_executor

Add the custom classification layer to it

In [6]:
# new layers
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
dense_layer_1 = tf.keras.layers.Dense(32, activation='elu')
prediction_layer = tf.keras.layers.Dense(3, activation = 'softmax')


# construct the final model
inputs = keras.Input(shape = (img_height, img_width, 3))
x = keras.applications.mobilenet_v3.preprocess_input(inputs)
x = base_model(x, training = False)
x = global_average_layer(x)
x = dense_layer_1(x)
#x = keras.layers.Dropout(0.05)(x)
outputs = prediction_layer(x)
model = keras.Model(inputs, outputs)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 40, 100, 3)]      0         
                                                                 
 MobilenetV3small (Function  (None, 2, 4, 576)         939120    
 al)                                                             
                                                                 
 global_average_pooling2d (  (None, 576)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 32)                18464     
                                                                 
 dense_1 (Dense)             (None, 3)                 99        
                                                                 
Total params: 957683 (3.65 MB)
Trainable params: 18563 (72.51

## 2.2 Start the initial learning process

Define Parameters for the learning process

In [7]:
base_learning_rate = 0.03
initial_epochs = 30

Compile the before built model

In [8]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

Train the model!

In [9]:
def scheduler(epoch, lr):
    if epoch == 0:
        return lr
    elif epoch % 5 == 0:
        return lr * 0.9 
    else: 
        return lr

In [10]:
#callback = keras.callbacks.LearningRateScheduler(scheduler)
callback = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001)

In [11]:
generator = keras.preprocessing.image.ImageDataGenerator()
batch_size = 32

history = model.fit(generator.flow(train_images, train_labels, batch_size), steps_per_epoch=len(train_images)/batch_size, validation_data=(validation_images, validation_labels), epochs=initial_epochs, callbacks=callback)

Epoch 1/30


2024-05-21 13:18:30.088165: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-05-21 13:18:30.927424: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f098546b1e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-21 13:18:30.927453: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 4GB Laptop GPU, Compute Capability 8.6
2024-05-21 13:18:30.931402: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1716290310.984423    8989 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


586/586 [==============================] - 10s 13ms/step - loss: 0.3863 - accuracy: 0.8598 - val_loss: 0.1991 - val_accuracy: 0.9321 - lr: 0.0300
Epoch 2/30
586/586 [==============================] - 6s 10ms/step - loss: 0.2415 - accuracy: 0.9085 - val_loss: 0.1958 - val_accuracy: 0.9327 - lr: 0.0300
Epoch 3/30
586/586 [==============================] - 6s 11ms/step - loss: 0.2248 - accuracy: 0.9162 - val_loss: 0.2179 - val_accuracy: 0.9205 - lr: 0.0300
Epoch 4/30
586/586 [==============================] - 6s 11ms/step - loss: 0.2310 - accuracy: 0.9178 - val_loss: 0.1481 - val_accuracy: 0.9481 - lr: 0.0300
Epoch 5/30
586/586 [==============================] - 7s 13ms/step - loss: 0.2081 - accuracy: 0.9249 - val_loss: 0.2110 - val_accuracy: 0.9233 - lr: 0.0300
Epoch 6/30
586/586 [==============================] - 9s 15ms/step - loss: 0.1931 - accuracy: 0.9305 - val_loss: 0.3051 - val_accuracy: 0.8753 - lr: 0.0300
Epoch 7/30
586/586 [==============================] - 9s 14ms/step - loss:

In [ ]:
model.evaluate(x = validation_images, y = validation_labels)

# 3. Fine tuning the base model

Freeze all layers except for the uppermost layers of the base model. Layzers will be frozen from ***fine_tune_from*** to ***len(base_model.layers)***.

In [12]:
base_model.trainable = True
fine_tune_from = len(base_model.layers) - 25

for layer in base_model.layers[:fine_tune_from]:
    layer.trainable = False

# don't train the dense layer
model.layers[len(model.layers) - 1].trainable = False
model.layers[len(model.layers) - 2].trainable = False

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 40, 100, 3)]      0         
                                                                 
 MobilenetV3small (Function  (None, 2, 4, 576)         939120    
 al)                                                             
                                                                 
 global_average_pooling2d (  (None, 576)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 32)                18464     
                                                                 
 dense_1 (Dense)             (None, 3)                 99        
                                                                 
Total params: 957683 (3.65 MB)
Trainable params: 294096 (1.12

Compile the model again but decrease the learning rate in order to avoid overfitting.

In [13]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate/100), loss='categorical_crossentropy', metrics=['accuracy'])

Train the uppermost layers of the base model and proceed where you left off in the initial training by using the last epoch of the first training as the initial epoch for fine tuning.

In [14]:
finetune_history = model.fit(generator.flow(train_images, train_labels, batch_size), steps_per_epoch=len(train_images)/batch_size, validation_data=generator.flow(validation_images, validation_labels, batch_size), initial_epoch=history.epoch[-1], epochs=initial_epochs + 10, callbacks=callback)

Epoch 30/40
586/586 [==============================] - 31s 36ms/step - loss: 0.1644 - accuracy: 0.9462 - val_loss: 0.1671 - val_accuracy: 0.9470 - lr: 3.0000e-04
Epoch 31/40
586/586 [==============================] - 14s 24ms/step - loss: 0.1217 - accuracy: 0.9590 - val_loss: 0.1504 - val_accuracy: 0.9498 - lr: 3.0000e-04
Epoch 32/40
586/586 [==============================] - 14s 24ms/step - loss: 0.1033 - accuracy: 0.9650 - val_loss: 0.1980 - val_accuracy: 0.9376 - lr: 3.0000e-04
Epoch 33/40
586/586 [==============================] - 14s 23ms/step - loss: 0.0926 - accuracy: 0.9686 - val_loss: 0.1848 - val_accuracy: 0.9404 - lr: 3.0000e-04
Epoch 34/40
586/586 [==============================] - 14s 24ms/step - loss: 0.0802 - accuracy: 0.9725 - val_loss: 0.1221 - val_accuracy: 0.9658 - lr: 3.0000e-04
Epoch 35/40
586/586 [==============================] - 14s 23ms/step - loss: 0.0726 - accuracy: 0.9757 - val_loss: 0.1277 - val_accuracy: 0.9625 - lr: 3.0000e-04
Epoch 36/40
586/586 [=======

In [ ]:
print(finetune_history.history.keys())
key = 'accuracy'
for loss in history.history[key]:
    print(loss, end='\r\n')
for loss in finetune_history.history[key]:
    print(loss, end='\r\n')

# 4. Save the model

Info: Loading can be done via: ***model = tf.keras.models.load_model(PATH_TO_MODEL_FILE)***


In [15]:
# for now until it is implemented properly: manually make sure that the path exists (in this case '/mnt/d/Dev/DHBW/DHBW_Studienarbeit/models/')
save_path = '/mnt/d/Dev/DHBW/DHBW_Studienarbeit/models/mn3_w100_h40_crop_14_contrast_grayscale_96_val_99_train.keras'

model.save(save_path)